In [39]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [40]:
import pandas as pd

#unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
ch_names = ['timestamps','Fp1','Fp2','C3','C4','P7','P8','O1','O2','F7','F8','F3','F4','T7','T8','P3','P4','stim','sfreq']
#unfiltered_subjects = ['Aanand_19_02.csv']
# Create an empty DataFrame to store all data
all_data = pd.DataFrame(columns=ch_names)

for file_name in unfiltered_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    all_data = pd.concat([all_data, combined_df])

C:\Users\LEGION\AppData\Local\Temp\ipykernel_3776\2273160114.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, combined_df])


In [41]:
drop1 = all_data.drop(['sfreq'], axis=1)
df = drop1.drop(['stim'], axis=1)


#drop_data = testing_data.drop(['sfreq'], axis=1)
# df2 = drop_data.drop(['stim'], axis=1)

In [42]:
df.shape

(636251, 18)

In [43]:
label_counts = df['label'].value_counts()
print(label_counts)

df1 = df

label
1.0    243767
2.0    211038
3.0    181446
Name: count, dtype: int64


In [44]:
df = df1.drop(['label'], axis=1)

In [45]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)
raw.set_eeg_reference()

raw.filter(l_freq = 1,h_freq = 30)

Creating RawArray with float64 data, n_channels=16, n_times=636251
    Range : 0 ... 636250 =      0.000 ...  5302.083 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.9s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [46]:
duration = 0.8
epochs=mne.make_fixed_length_epochs(raw, duration = duration, overlap=0,preload=True)

Not setting metadata
6627 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6627 events and 96 original time points ...
0 bad epochs dropped


In [47]:
epochs.drop_bad()  # This will drop bad epochs
num_epochs = len(epochs)
print("Total number of epochs created:", num_epochs)

Total number of epochs created: 6627


In [48]:
epochs.get_data()

C:\Users\LEGION\AppData\Local\Temp\ipykernel_3776\1251544707.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs.get_data()


array([[[ 1.81898940e-11, -9.47492918e+04, -1.18740849e+05, ...,
         -1.55660446e+01,  2.59795857e+02,  9.72787301e+02],
        [ 3.43334250e-11, -8.71324937e+04, -1.09063184e+05, ...,
         -2.14616536e+02,  1.49394612e+02,  9.72879558e+02],
        [-1.20508048e-11, -4.18075628e+03, -3.32758466e+03, ...,
         -4.30897907e+02, -3.79045729e+02,  7.76375694e+01],
        ...,
        [ 7.27595761e-12,  1.05012965e+06,  1.34087222e+06, ...,
         -4.24922484e+03, -7.47103394e+03, -1.05826706e+04],
        [ 2.27373675e-12,  2.93703433e+04,  3.14944841e+04, ...,
          2.54130906e+02,  1.94539450e+02, -6.54052595e+02],
        [ 8.73114914e-11, -5.94392622e+05, -7.53669296e+05, ...,
          3.24470825e+03,  4.84234105e+03,  6.34271940e+03]],

       [[ 1.54341874e+03,  1.44978755e+03,  1.09827221e+03, ...,
         -1.70765203e+02,  2.10130711e+02,  5.63409234e+02],
        [ 1.48781240e+03,  1.24182125e+03,  8.26740544e+02, ...,
         -1.78832822e+02,  3.04379923e

In [49]:
import mne

eeg_data = epochs.get_data()

condition_ranges = {'lev1': (0, (num_epochs/3)),
                    'lev2': ((num_epochs/3) , 2*((num_epochs)/3) ),
                    'lev3': ( 2*((num_epochs)/3) , (num_epochs) )}

epochs_by_condition = {}
selected_epochs = []

for condition, (start_epoch, end_epoch) in condition_ranges.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = epochs[start_epoch:end_epoch+1].copy()
    epochs_by_condition[condition] = condition_epochs
    selected_epochs.append(condition_epochs)

selected_epochs = mne.concatenate_epochs(selected_epochs)

for condition, epoch_data in epochs_by_condition.items():
    num_epochs = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs}")

print(f"Total number of selected epochs: {len(selected_epochs)}")


C:\Users\LEGION\AppData\Local\Temp\ipykernel_3776\3628746636.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Not setting metadata
6629 matching events found
No baseline correction applied
Number of epochs for lev1: 2210
Number of epochs for lev2: 2210
Number of epochs for lev3: 2209
Total number of selected epochs: 6629


In [50]:

condition_of_interest = 'lev1'
epoch_array_for_condition = epochs_by_condition[condition_of_interest]
print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


Epoch array shape for condition lev1: (2210, 16, 96)


C:\Users\LEGION\AppData\Local\Temp\ipykernel_3776\1489624503.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


In [51]:
import pandas as pd
import numpy as np
from scipy.signal import welch



def calculate_psd_features(epoch_data, sfreq):
    psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
    psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
    psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
    psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

    freq_min = 14
    freq_max = 30

    # Find the indices corresponding to the frequency range of interest
    idx_min = np.argmax(psd_frequencies > freq_min) - 1
    idx_max = np.argmax(psd_frequencies > freq_max)

    # Extract the PSD within the specified frequency range
    psd_values_alpha_band = psd_values[:, idx_min:idx_max]
    psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
    psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
    psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
    return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band

channel_names = epochs.ch_names
sfreq = epochs.info['sfreq']  # Sampling frequency

columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
columns.append('condition')
feature_df = pd.DataFrame(columns=columns)

def append_epoch_data(epoch_array, condition_label):
    global feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=columns)
        feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

append_epoch_data(epochs_by_condition['lev1'], 'lev1')
append_epoch_data(epochs_by_condition['lev2'], 'lev2')
append_epoch_data(epochs_by_condition['lev3'], 'lev3')

print("Shape of feature DataFrame:", feature_df.shape)
print(feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 96, using nperseg = 96
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of feature DataFrame: (6629, 49)
        mean_psd_Fp1       mean_psd_Fp2         mean_psd_C3  \
0   5694.22360266334  7146.717077465267   4649.465088410274   
1  4586.673753313572    6559.3241362172  4165.4277034024335   
2  4625.831904840678  5650.923485871639  3794.7110021388335   
3  2978.774481838965  4145.629188683513  2948.6676668442756   
4  3477.818521894308  5214.105099272281   3211.650303027604   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   2696.568564688343   6257.966056537798   5029.379662283178   
1   1972.899916415364  4650.7038966766195   4407.191188658524   
2  2158.9866755250446  4016.4541572858047   3975.366711483707   
3  1633.5500949320788   3099.690584331302  2616.2899138308953   
4  1703.9709405741703   4560.881219513765  3613.3045723806404   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   5564.876570219283    4669.74713156826  3234.3550607194666   
1    6498.98990230998  3785.1386794447862   3072.990643613976

In [52]:
# Rename columns using a dictionary
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

# Rename columns using the rename() method
feature_df.rename(columns=new_column_names, inplace=True)

# Print the DataFrame
print(feature_df.head())


        mean_psd_Fp1       mean_psd_Fp2         mean_psd_C3  \
0   5694.22360266334  7146.717077465267   4649.465088410274   
1  4586.673753313572    6559.3241362172  4165.4277034024335   
2  4625.831904840678  5650.923485871639  3794.7110021388335   
3  2978.774481838965  4145.629188683513  2948.6676668442756   
4  3477.818521894308  5214.105099272281   3211.650303027604   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   2696.568564688343   6257.966056537798   5029.379662283178   
1   1972.899916415364  4650.7038966766195   4407.191188658524   
2  2158.9866755250446  4016.4541572858047   3975.366711483707   
3  1633.5500949320788   3099.690584331302  2616.2899138308953   
4  1703.9709405741703   4560.881219513765  3613.3045723806404   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   5564.876570219283    4669.74713156826  3234.3550607194666   
1    6498.98990230998  3785.1386794447862   3072.990643613976   
2  4234.0536279225935  3305.3102567

In [53]:
feature_df

,mean_psd_Fp1,mean_psd_Fp2,mean_psd_C3,mean_psd_C4,mean_psd_P7,mean_psd_P8,mean_psd_O1,mean_psd_O2,mean_psd_F7,mean_psd_F8,...,stddev_psd_O2,stddev_psd_F7,stddev_psd_F8,standarddev_psd_F3,standarddev_psd_F4,standarddev_psd_T7,standarddev_psd_T8,standarddev_psd_P3,standarddev_psd_P4,condition
0,5694.22360266334,7146.717077465267,4649.465088410274,2696.568564688343,6257.966056537798,5029.379662283178,5564.876570219283,4669.74713156826,3234.3550607194666,10426.299898035892,...,10849.996873131737,7031.019869862794,24426.771396370576,28196.5956114869,3818.3774525535327,27532.284852330988,6033.481325755221,29307.24322618626,5367.8936126925555,lev1
1,4586.673753313572,6559.3241362172,4165.4277034024335,1972.899916415364,4650.7038966766195,4407.191188658524,6498.98990230998,3785.1386794447862,3072.990643613976,7700.85980634766,...,8457.953764144959,5855.767940449178,17198.643998182844,21271.528579135804,3834.14927927274,20942.443966029707,2580.8606328575142,21657.928233724768,2329.2775237326646,lev1
2,4625.831904840678,5650.923485871639,3794.7110021388335,2158.9866755250446,4016.4541572858047,3975.366711483707,4234.0536279225935,3305.3102567510655,2794.16246881082,6687.840823696272,...,7667.55899133999,5860.621162579309,15418.249359006011,18552.79387807523,3566.4330214549063,18916.559652277178,1887.9234435828687,19227.62659230712,1943.4572677659241,lev1
3,2978.774481838965,4145.629188683513,2948.6676668442756,1633.5500949320788,3099.690584331302,2616.2899138308953,4980.316777429168,2237.895615366025,1899.5602244566617,6096.312858627338,...,5263.483810358813,3339.678346387836,14044.148153231685,17428.059832572504,1952.562281629209,17339.601154533484,1938.091315966789,18371.163087549376,2065.1830587296618,lev1
4,3477.818521894308,5214.105099272281,3211.650303027604,1703.9709405741703,4560.881219513765,3613.3045723806404,6814.527459214364,3533.593278457455,3067.124190450562,6133.618905952871,...,7764.712824564605,5744.382564708871,14137.19570219287,16976.54179666338,3721.0773698060793,16585.581000866307,2448.917141377944,17317.968382732113,2574.3850939315116,lev1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,1308.569692489256,527.5129668415118,401.2651820800487,498.7806124558345,381.16615963690253,496.7654049399388,371.74726462088717,384.07213452827676,1258.9945654886656,134.0875840891292,...,407.3210205582035,867.7678319388469,127.05463672878454,124.3448612590621,287.2906968968782,134.00695574723326,180.2492408796621,163.3813410829104,383.7073269066589,lev3
6625,1277.7299093074128,520.3045814598171,503.76023366963125,435.3451352784383,421.34077531386936,436.58613952846935,446.54526878045425,575.9751677246719,864.0335198249037,347.2850892741217,...,635.4857744591473,778.32362074155,321.7495722623806,197.53967483052097,320.230810852122,147.51628297701026,175.67020300690976,187.360727701153,347.67969032190814,lev3
6626,20379.609932265364,6097.3178118646765,5902.168028691973,6063.715738635613,5698.771252851492,5981.742582858519,5804.111480001088,6033.373800894983,5006.235779773802,1944.880662690435,...,7126.793364395995,6294.795565692156,2132.486323134326,1900.382869467494,4343.300760212668,2239.0617102412016,1975.9134115568038,2183.9226992905697,5227.424907480367,lev3
6627,270003.971414891,13943.894155795775,37042.98945202232,37021.744716521825,20123.009030180754,9191.31526346621,31554.861637114933,43402.06876847074,357641.45893537224,5424.633160293224,...,10629.38771066282,226477.71722669504,5565.68000952413,3797.76621897224,37758.65586509981,4266.852494898786,18388.070515054227,6188.642393152062,9919.83806829047,lev3


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9245852187028658


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

gb_classifier.fit(X_train, y_train)

y_pred = gb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9049773755656109


In [58]:
import numpy as np
import pandas as pd

# Assuming X_test and y_test are NumPy arrays or pandas DataFrames

# Saving X_test
np.save('X_test.npy', X_test)  # if X_test is a NumPy array
# or
X_test.to_csv('X_test.csv', index=False)  # if X_test is a pandas DataFrame

# Saving y_test
np.save('y_test.npy', y_test)  # if y_test is a NumPy array
# or
pd.DataFrame(y_test).to_csv('y_test.csv', index=False, header=False)  # if y_test is a pandas Series

In [56]:
# def append_epoch_data(epoch_array, condition_label):
#         global feature_df
#         for epoch in epoch_array:
#             psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
#             epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
#             epoch_features = np.append(epoch_features, condition_label)
#             epoch_df = pd.DataFrame([epoch_features], columns=columns)
#             feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

# def calculate_psd_features(epoch_data, sfreq):
#         psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
#         psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
#         psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
#         psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

#         freq_min = 14
#         freq_max = 30

#         # Find the indices corresponding to the frequency range of interest
#         idx_min = np.argmax(psd_frequencies > freq_min) - 1
#         idx_max = np.argmax(psd_frequencies > freq_max)

#         # Extract the PSD within the specified frequency range
#         psd_values_alpha_band = psd_values[:, idx_min:idx_max]
#         psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
#         psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
#         psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
#         return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band


In [57]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     rf_classifier.fit(X_train, y_train)
#     y_pred = rf_classifier.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()
